In [ ]:
# Reference: https://github.com/MahaQ3/A_Conversational_Chatbot_model/blob/main/chatbot.ipynb
# Reference: Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]Retrieved: https://nlp.stanford.edu/projects/glove/

In [ ]:


from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# prompt: how to improve encoding below

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed
from tensorflow.keras.models import Model
from IPython.core.display import display, HTML
from tensorflow.keras.layers import TimeDistributed, Dense
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import re
import nltk
from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model
from keras.layers import Activation, dot, concatenate
from google.colab import drive
# Reference: https://github.com/MahaQ3/A_Conversational_Chatbot_model/blob/main/chatbot.ipynb
# Reference: Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]Retrieved: https://nlp.stanford.edu/projects/glove/


INPUT_LENGTH = 20
OUTPUT_LENGTH = 20


txt=open('/content/drive/MyDrive/movie-corpus/movie_lines_pre_processed_for_test.tsv','r').readlines()
# Clean data

def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", text)
    text = " ".join(text.split())
    return text

# Print the first few lines of the text data after cleaning
for i in range(5):
  print(f"Original Text: {txt[i]}")
  print(f"Cleaned Text: {clean_text(txt[i])}")
  print("---")


#split the data to questions and answers
questions=[]
answers=[]
for index,sent in enumerate(txt):
  if index%2==0:
    questions.append(sent)
  else:
    answers.append(sent)


print(f"Number of questions: {len(questions)}")
print(f"Number of answers: {len(answers)}")

if len(questions) > 0 and len(answers) > 0:
  print("The code worked successfully!")
  print("Questions and answers have been split.")
else:
  print("The code may not have worked correctly.")
  print("Questions or answers are empty.")


nltk.download('punkt')
#choosing number of samples
num_samples = 30000  # Number of samples to train on.

# Initialize short_questions and short_answers by slicing the original lists:
short_questions = questions[:num_samples] # Slicing the 'questions' list
short_answers = answers[:num_samples]   # Slicing the 'answers' list

#tokenizing the qns and answers
short_questions_tok = [nltk.word_tokenize(sent) for sent in short_questions]
short_answers_tok = [nltk.word_tokenize(sent) for sent in short_answers]

# training data & validation data


#train-validation split
data_size = len(short_questions_tok)

# We will use the first 0-80th %-tile (80%) of data for the training
training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance
training_output = short_answers_tok[:round(data_size*(80/100))]

# We will use the remaining for validation
validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))
# Word en/decoding dictionaries


# Create a dictionary for the frequency of the vocabulary
vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab


Original Text: L49	u0	m0	Did you change your hair?	No.	change hair	1

Cleaned Text: l49 u0 m0 did you change your hair? no. change hair 1
---
Original Text: L59	u9	m0	I missed you.	It says here you exposed yourself to a group of freshmen girls.	miss	0

Cleaned Text: l59 u9 m0 i missed you. it says here you exposed yourself to a group of freshmen girls. miss 0
---
Original Text: L60	u8	m0	It says here you exposed yourself to a group of freshmen girls.	It was a bratwurst. I was eating lunch.	say expose group freshman girl	0

Cleaned Text: l60 u8 m0 it says here you exposed yourself to a group of freshmen girls. it was a bratwurst. i was eating lunch. say expose group freshman girl 0
---
Original Text: L61	u9	m0	It was a bratwurst. I was eating lunch.	With the teeth of your zipper?	bratwurst eat lunch	0

Cleaned Text: l61 u9 m0 it was a bratwurst. i was eating lunch. with the teeth of your zipper? bratwurst eat lunch 0
---
Original Text: L63	u7	m0	You the new guy?	So they tell me...	new g

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


training size 12000
validation size 3000


In [ ]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 15
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 40808
Size of vocab we will use: 2795


In [ ]:
#we will create dictionaries to provide a unique integer for each word.
WORD_CODE_START = 1
WORD_CODE_END=2
WORD_CODE_PADDING = 0

word_num  = 3 #number 1 & 2  are left for WORD_CODE_START & WORD_CODE_END for model decoder later
encoding = {'START': 1, 'END':2}
decoding = {1:  'START',2: 'END'}
for word, count in vocab.items():
    if count >= threshold: #get vocabularies that appear above threshold count
        encoding[word] = word_num
        decoding[word_num ] = word
        word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 2798


In [ ]:
#include unknown token for words not in dictionary
decoding[len(encoding)+3] = '<UNK>'
encoding['<UNK>'] = len(encoding)+3

In [ ]:
dict_size = word_num+3


In [ ]:

np.save('word2id.npy',encoding)
np.save('id2word.npy',decoding)

In [ ]:
def transform(encoding, data, vector_size=20):
    """
    :param encoding: encoding dict built by build_word_encoding()
    :param data: list of strings
    :param vector_size: size of each encoded vector
    """
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                np.zeros(shape=(len(data), vector_size))[i][j] = encoding[data[i][j]]
            except:
                np.zeros(shape=(len(data), vector_size))[i][j] = encoding['<UNK>']
    return np.zeros(shape=(len(data), vector_size))

In [ ]:
tf.keras.backend.clear_session()

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed
from tensorflow.keras.models import Model


In [ ]:
# Define input layers with unique names
encoder_input = Input(shape=(INPUT_LENGTH,), name='encoder_input')
decoder_input = Input(shape=(INPUT_LENGTH,), name='decoder_input')

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/MyDrive/glove.twitter.27B.25d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((dict_size, 25))
for word, i in encoding.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

embed_layer = Embedding(input_dim=dict_size, output_dim=25,input_length=INPUT_LENGTH, trainable=True, mask_zero=True)
embed_layer.build((None,))
embed_layer.set_weights([embedding_matrix])

Loaded 1193514 word vectors.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
encoder = embed_layer(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:, -1, :]

print('encoder', encoder)
print('encoder_last', encoder_last)

# Use the existing decoder_input
decoder_embedding = embed_layer(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder_embedding, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

# For the plain Sequence-to-Sequence, we produced the output from directly from decoder
# output = TimeDistributed(Dense(output_dict_size,
# activation="softmax"))(decoder)

encoder <KerasTensor shape=(None, 20, 512), dtype=float32, sparse=False, name=keras_tensor_2>
encoder_last <KerasTensor shape=(None, 512), dtype=float32, sparse=False, name=keras_tensor_3>
decoder <KerasTensor shape=(None, 20, 512), dtype=float32, sparse=False, name=keras_tensor_6>


In [ ]:
!pip install keras-self-attention
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, activations, regularizers, constraints
from tensorflow.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, units, activation='tanh',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(AttentionLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=6b302204b5564816850523557e63f1ac80c5526e6ae55a4b88f41aba8af26932
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras-self-attention


In [ ]:
import tensorflow as tf

class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.units = units
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, inputs):
        decoder_hidden_state, encoder_outputs = inputs

        # Expand dims to enable broadcasting
        decoder_hidden_state_time = tf.expand_dims(decoder_hidden_state, 1)

        # Calculate score using dot-product attention
        score = self.V(tf.tanh(
            self.W1(decoder_hidden_state_time) + self.W2(encoder_outputs)))

        # Apply softmax to get attention weights
        attention_weights = tf.nn.softmax(score, axis=1)

        # Calculate context vector
        context_vector = attention_weights * encoder_outputs
        context_vector = tf.reduce_sum(context_vector, axis=1)

        # Concatenate context vector and decoder hidden state
        output = tf.keras.layers.concatenate([context_vector, decoder_hidden_state], axis=-1) #  concatenate on the last axis

        # # Apply TimeDistributed Dense layer to the output (optional)
        # output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(self.units, activation="tanh"))(output)
        return output



In [ ]:
import tensorflow as tf

class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.units = units
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, inputs):
        decoder_hidden_state, encoder_outputs = inputs

        # Expand dims to enable broadcasting
        decoder_hidden_state_time = tf.expand_dims(decoder_hidden_state, 1)

        # Calculate score using dot-product attention
        score = self.V(tf.tanh(
            self.W1(decoder_hidden_state_time) + self.W2(encoder_outputs)))

        # Apply softmax to get attention weights
        attention_weights = tf.nn.softmax(score, axis=1)

        # Calculate context vector
        context_vector = attention_weights * encoder_outputs
        context_vector = tf.reduce_sum(context_vector, axis=1)

        # Concatenate context vector and decoder hidden state
        output = tf.keras.layers.concatenate([context_vector, decoder_hidden_state], axis=-1) #  concatenate on the last axis

        # # Apply TimeDistributed Dense layer to the output (optional)
        # output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(self.units, activation="tanh"))(output)
        return output

import tensorflow as tf

# Alternatively if you have a decoding dictionary
# Replace `decoding` with your decoding dictionary
target_vocabulary = list(decoding.keys())

# Calculate the output vocabulary size based on your target vocabulary
output_dict_size = len(target_vocabulary)  # or however you define the output vocabulary size

# Define the output layer
output = tf.keras.layers.TimeDistributed(
    tf.keras.layers.Dense(output_dict_size, activation="softmax")
)(decoder)  # or your attention mechanism output

# Create the model
model = tf.keras.Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ decoder_input             │ (None, 20)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_input             │ (None, 20)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 20, 25)         │         70,025 │ encoder_input[0][0],   │
│                           │                        │                │ decoder_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 20)             │              0 │ encoder_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 20, 512)        │      1,101,824 │ embedding[0][0],       │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 512)            │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 20, 512)        │      1,101,824 │ embedding[1][0],       │
│                           │                        │                │ get_item[0][0],        │
│                           │                        │                │ get_item[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_1 (NotEqual)    │ (None, 20)             │              0 │ decoder_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, 20, 2798)       │      1,435,374 │ lstm_1[0][0],          │
│ (TimeDistributed)         │                        │                │ not_equal_1[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,709,047 (14.15 MB)

 Trainable params: 3,709,047 (14.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_batch(input_data, output_text_data, batch_size):
    while True:
        for j in range(0, len(input_data), batch_size):
            # Pad both encoder and decoder input data to INPUT_LENGTH (20)
            encoder_input_data = pad_sequences(input_data[j:j + batch_size], INPUT_LENGTH).astype('float32')
            decoder_input_data = pad_sequences(output_text_data[j:j + batch_size], INPUT_LENGTH).astype('float32')
            decoder_target_data = np.zeros((len(output_text_data[j:j + batch_size]), INPUT_LENGTH, dict_size), # Change target data shape to match
                                          dtype="float32")

            for i, (input_text, target_text) in enumerate(zip(output_text_data[j:j + batch_size], output_text_data[j:j + batch_size])):
                for t, word in enumerate(target_text):
                    if t > 0:
                        if word in encoding and encoding[word] < dict_size:
                            decoder_target_data[i, t - 1, encoding[word]] = 1
                        else:
                            if '<UNK>' in encoding and encoding['<UNK>'] < dict_size:
                                decoder_target_data[i, t - 1, encoding['<UNK>']] = 1
                            else:
                                pass

            yield (encoder_input_data, decoder_input_data), decoder_target_data

INPUT_LENGTH = 20
#OUTPUT_LENGTH = 22 # Remove or adjust if needed but keep consistent
dict_size = 1000

# Define output types for the generator
output_types = ((tf.float32, tf.float32), tf.float32)  # Tuple of types for (encoder_input, decoder_input), decoder_target

# Define output shapes for the generator, use INPUT_LENGTH consistently
output_shapes = (
    (tf.TensorShape([None, INPUT_LENGTH]), tf.TensorShape([None, INPUT_LENGTH])),  # Shapes for encoder_input, decoder_input
    tf.TensorShape([None, INPUT_LENGTH, dict_size])  # Shape for decoder_target
)

# Create the dataset using from_generator with output_types and output_shapes
train_gen = tf.data.Dataset.from_generator(
    lambda: generate_batch(encoded_training_input, encoded_training_output, 128),
    output_types=output_types,
    output_shapes=output_shapes
).prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_batch(input_data, output_text_data, batch_size):
    while True:
        for j in range(0, len(input_data), batch_size):
            # Pad both encoder and decoder input data to INPUT_LENGTH (20)
            encoder_input_data = pad_sequences(input_data[j:j + batch_size], INPUT_LENGTH).astype('float32')
            decoder_input_data = pad_sequences(output_text_data[j:j + batch_size], INPUT_LENGTH).astype('float32')
            decoder_target_data = np.zeros((len(output_text_data[j:j + batch_size]), INPUT_LENGTH, dict_size), # Change target data shape to match
                                          dtype="float32")

            for i, (input_text, target_text) in enumerate(zip(output_text_data[j:j + batch_size], output_text_data[j:j + batch_size])):
                for t, word in enumerate(target_text):
                    if t > 0:
                        if word in encoding and encoding[word] < dict_size:
                            decoder_target_data[i, t - 1, encoding[word]] = 1
                        else:
                            if '<UNK>' in encoding and encoding['<UNK>'] < dict_size:
                                decoder_target_data[i, t - 1, encoding['<UNK>']] = 1
                            else:
                                pass

            # Create a mask manually and convert to tf.Tensor
            mask = tf.convert_to_tensor(encoder_input_data != 0, dtype=tf.float32)

            yield (encoder_input_data, decoder_input_data, mask), decoder_target_data # Yield the mask as part of the input data


# Update the model's input to include the mask
encoder_input = tf.keras.Input(shape=(INPUT_LENGTH,), name='encoder_input')
decoder_input = tf.keras.Input(shape=(INPUT_LENGTH,), name='decoder_input')
mask_input = tf.keras.Input(shape=(INPUT_LENGTH,), name='mask_input')  # Add mask input

# Update the model's inputs to include the mask
model = tf.keras.Model(inputs=[encoder_input, decoder_input, mask_input], outputs=[output])  # Include mask in inputs
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


model.fit(
    train_gen,  # Make sure train_gen yields (encoder_input, decoder_input, mask), decoder_target
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    verbose=1,
    validation_data=val_gen,  # Make sure val_gen yields (encoder_input, decoder_input, mask), decoder_target
    validation_steps=validation_steps
)

ValueError: The name "decoder_input" is used 2 times in the model. All operation names should be unique.